<a href="https://colab.research.google.com/github/MitchellKrops/PythonAutomation/blob/main/Level_Product_converter.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Importeerassistent voor maatschappijproducten

In [4]:
#@title Default title text
##@title Importeerassistent voor maatschappij producten
## Importeer XML lib
import xml.etree.ElementTree as ET
import os
from tabulate import tabulate
from google.colab import files

def main(Bron_Bestand):
  tree = ET.parse(Bron_Bestand)
  root = tree.getroot()
  print("-----------------------------------------------------------------------------")

  Branchecodes_list = []
  EXTBRA_list = []
  waarde = root.find('INFO').text
  for Naam_EXTPRD in root.iter("NM_EXTPRD"):
    Externe_productomschrijving = Naam_EXTPRD.text
  infotext = waarde.split("-")
  Bron_Maatschappij = infotext[1]
  Soort_Product = infotext[2]
  Productcode = infotext[3]
  MIJ_SRT_VRZ = root.findall('.//MIJ_SRT_VRZ')
  for MIJ_SRT in MIJ_SRT_VRZ :
    if MIJ_SRT.find('NM_EXTBRA').text is not None:
      naam = MIJ_SRT.find('NM_EXTBRA').text
      EXTBRA_list.append(naam)
    else:
      naam = "**Niet ingevuld**"
      EXTBRA_list.append(naam)   
    br_code = MIJ_SRT.find('KD_BR_VAN').text
    Branchecodes_list.append(br_code)
  
  ## Tonen brongegevens
  print()
  if Bron_Bestand == 'temp.xml':
    print("Het exportproduct bevat de volgende gegevens:")
  else:  
    print("Het bronproduct bevat de volgende gegevens:")
  
  print()
  info_data = [[Bron_Maatschappij, Soort_Product, Productcode, Externe_productomschrijving]]
  print (tabulate(info_data, headers=["Maatschappij", "Soort product", "Productcode","Externe productomschrijving"],numalign="left"))
  print()
    
  branche_data = zip(Branchecodes_list,EXTBRA_list)
  print (tabulate(branche_data, headers=["Branchecode","Externe brancheomschrijving"],numalign="left"))
  print()
  print("-----------------------------------------------------------------------------")
  ## Input doelgegevens
  if Bron_Bestand != 'temp.xml':
    print("Voer nieuwe waarden in (enter is oude waarde)")
    print()
    while True:
      global Doel_Maatschappij
      Doel_Maatschappij = input("Geef doel maatschappij in:") or Bron_Maatschappij
      if not len(Doel_Maatschappij) <= 5:
        print("**Fout: Maatschappijnaam mag maximaal 5 karakters lang zijn**")
        continue
      else:

        break  

    while True:  
      global Doel_Productcode
      Doel_Productcode = input("Geef doel productcode in:") or Productcode
      if len(Doel_Productcode) > 3 or Doel_Productcode.isdecimal() == False:
        print("**Fout: Productcode mag maximaal 3 cijfers lang zijn**")
        continue
      else:

        break 
    print()
    Naam_EXTPRD_keuze = input("Wil je de externe productomschrijving aanpassen? y/n:")
    Branchecodes_keuze = input("Wil je branchecodes aanpassen? y/n:")
    Branchecode_omschrijving_keuze = input("Wil je de externe brancheomschrijving aanpassen? y/n:")
    print()  
    if Naam_EXTPRD_keuze == "y":
      while True:
        Naam_EXTPRD_nieuw = input("Geef nieuwe externe productomschrijving in:")
        if len(Naam_EXTPRD_nieuw) > 50:
          print("**Fout: Externe productomschrijving mag maximaal 50 karakters lang zijn**")
          continue
        else:
          print()
          for element in root.iter('NM_EXTPRD'):
            element.text = Naam_EXTPRD_nieuw
          break
  
    ## Aanmaken nieuw INFO Attribute
    Info_Nieuw = "-{0}-{1}-{2}".format(Doel_Maatschappij, Soort_Product, Doel_Productcode)

    ## Wegschrijven nieuw INFO Attribute
    for element in root.iter('INFO'):
        element.text = Info_Nieuw

    for productcode in root.iter('KD_PROD'):
        productcode.text = productcode.text.replace(Productcode, Doel_Productcode)

    for element in root.iter():
        try:
            element.text = element.text.replace(Bron_Maatschappij, Doel_Maatschappij)
        except AttributeError:
            pass

    ## Vastleggen en tonen branchecodes / dekkingscodes
    Branchecodes_list_nieuw = []
    if Branchecodes_keuze == "y":
        for i in Branchecodes_list:
            
            while True:
              branchecode_nieuw = input("Branchecode "+ i + " vervangen door:") or i
              if len(branchecode_nieuw) > 3 or branchecode_nieuw.isdecimal() == False:
                print("**Fout: Branchecode mag maximaal 3 cijfers lang zijn**")
                continue
              else:

                break
            Branchecodes_list_nieuw.append(branchecode_nieuw)
        print()
        ## Vervangen van brongegevens voor doelgegevens
        Branchecodes_rubrieken = ['KD_BR_TOT', 'KD_BR_VAN', 'KD_BR_ELM']
        for i in range(len(Branchecodes_rubrieken)):
            for element in root.iter(Branchecodes_rubrieken[i]):
                index = Branchecodes_list.index(element.text)
                element.text = Branchecodes_list_nieuw[index]
                
    EXTBRA_list_nieuw = []
    if Branchecode_omschrijving_keuze == "y":
      
      for i in EXTBRA_list:
        while True:
          EXTBRA_nieuw = input("Externe omschrijving "+ i + " vervangen door:") or i
          if len(EXTBRA_nieuw) > 50:
            print("**Fout: Externe omschrijving mag maximaal 50 karakters lang zijn")
            continue
          else:
            EXTBRA_list_nieuw.append(EXTBRA_nieuw) 
            break
          
      for element in root.iter('NM_EXTBRA'):
        if element.text is None:
          index = EXTBRA_list.index('**Niet ingevuld**')
          element.text = EXTBRA_list_nieuw[index]
        else:  
          index = EXTBRA_list.index(element.text)
          element.text = EXTBRA_list_nieuw[index]
          
      ##Info wegschrijven in tijdelijk bestand
    tree.write("temp.xml", method='xml', encoding='utf-8', xml_declaration=True)
    if input("Exportproduct tonen? y/n:") == "y":
      main("temp.xml")
    else:
      tree.write("{0}-{1}-{2}.xml".format(Doel_Maatschappij, Soort_Product, Doel_Productcode), method='xml', encoding='utf-8', xml_declaration=True)
      print()
      print("Succesvol importbestand aangemaakt: " + Doel_Maatschappij + "-" + Soort_Product + "-" + Doel_Productcode + ".xml")

      
  else:
    if input("Akkoord? y/n:") == 'y':
      ## Informatie wegschrijven in Export.xml
      tree.write("{0}-{1}-{2}.xml".format(Doel_Maatschappij, Soort_Product, Doel_Productcode), method='xml', encoding='utf-8', xml_declaration=True)
      os.remove("temp.xml")
      print()
      a = input("Succesvol importbestand aangemaakt: " + Doel_Maatschappij + "-" + Soort_Product + "-" + Doel_Productcode + ".xml, direct downloaden ? y/n:")
      if a == 'y':
        files.download(Doel_Maatschappij + "-" + Soort_Product + "-" + Doel_Productcode + ".xml") 
      
    else:
      main('temp.xml')

if __name__ == '__main__':
    try:
        bestand = input("Geef bronbestand in:")
        main(bestand)
    except KeyboardInterrupt:
        print('Programma gestopt')


Geef bronbestand in:Bron.xml
-----------------------------------------------------------------------------

Het bronproduct bevat de volgende gegevens:

Maatschappij    Soort product    Productcode    Externe productomschrijving
--------------  ---------------  -------------  -----------------------------
RCCLA           AB               300            Aansprakelijkheid MKB

Branchecode    Externe brancheomschrijving
-------------  -----------------------------
100            **Niet ingevuld**
110            Eigenaarsrisico
120            Omgevingsschade
130            Werkgeversaansprakelijkheid
420            Installatiewerk

-----------------------------------------------------------------------------
Voer nieuwe waarden in (enter is oude waarde)

Geef doel maatschappij in:ZZZVP
Geef doel productcode in:878

Wil je de externe productomschrijving aanpassen? y/n:n
Wil je branchecodes aanpassen? y/n:n
Wil je de externe brancheomschrijving aanpassen? y/n:n

Exportproduct tonen? y/n:y
--

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>